##### ARTI 560 - Computer Vision  
## Image Classification using Transfer Learning - Exercise 

### Objective

In this exercise, you will:

1. Select another pretrained model (e.g., VGG16, MobileNetV2, or EfficientNet) and fine-tune it for CIFAR-10 classification.  
You'll find the pretrained models in [Tensorflow Keras Applications Module](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

2. Before training, inspect the architecture using model.summary() and observe:
- Network depth
- Number of parameters
- Trainable vs Frozen layers

3. Then compare its performance with ResNet and the custom CNN.

### Questions:

- Which model achieved the highest accuracy?
- Which model trained faster?
- How might the architecture explain the differences?

depth is num of tranable layers

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB7 
from tensorflow.keras.applications.efficientnet import preprocess_input

# -----------------------------
# 1) Load CIFAR-10
# -----------------------------
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

class_names = [
    "airplane","automobile","bird","cat","deer",
    "dog","frog","horse","ship","truck"
]

# Keep labels as integers (SparseCategoricalCrossentropy)
y_train = y_train.squeeze().astype("int64")
y_test  = y_test.squeeze().astype("int64")

# Convert images to float32
x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")

# -----------------------------
# 2) Data augmentation
# -----------------------------
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [3]:
effnet_base= keras.applications.EfficientNetB7(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)
effnet_base.trainable = False  # freeze first (feature extractor)

# -----------------------------
# 4) Full model (preprocess inside model)
# -----------------------------
effnet_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(224, 224, interpolation="bilinear"),
    layers.Lambda(preprocess_input),     
    effnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10)                          # logits
], name="cifar10_EfficientNetB7")

effnet_model.summary()



258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


Model: "cifar10_EfficientNetB7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb7 (Functional)     │ (None, 7, 7, 2560)     │    64,097,687 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2560)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        25,610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,123,297 (244.61 MB)

 Trainable params: 25,610 (100.04 KB)

 Non-trainable params: 64,097,687 (244.51 MB)

---

In [8]:
# Top-level layers (what summary shows)
print("Top-level layers:", len(effnet_model.layers))

# True backbone depth
print("Backbone layers :", len(effnet_base.layers))

# Total true depth
print("Total depth     :", len(effnet_model.layers) + len(effnet_base.layers))


Top-level layers: 6
Backbone layers : 814
Total depth     : 820


### Observation:
#### - Network depth
The depth of the network is 6 layers ( i guess it should be 820 layers total, with 814 inside the backbone)
#### - Number of parameters
The total number of parameter is 64,123,297 (25,610 Trainable params, 64,097,687 Non-trainable params).
#### - Trainable vs Frozen layers
Trainable params : 25,610
Non-trainable params: 64,097,687
this means that EfficientNetB7 is completely frozen, and only the final Dense layer is being trained

---

In [4]:
# -----------------------------
# 5) Compile + Train (frozen backbone)
# -----------------------------
effnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1),
]

history = effnet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 703s 949ms/step - accuracy: 0.7416 - loss: 0.8386 - val_accuracy: 0.9122 - val_loss: 0.2829 - learning_rate: 0.0010
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 658s 935ms/step - accuracy: 0.8285 - loss: 0.5102 - val_accuracy: 0.9200 - val_loss: 0.2533 - learning_rate: 0.0010
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 658s 934ms/step - accuracy: 0.8393 - loss: 0.4726 - val_accuracy: 0.9202 - val_loss: 0.2496 - learning_rate: 0.0010


In [5]:

# -----------------------------
# 6) Test / Evaluate
# -----------------------------
test_loss, test_acc_r = effnet_model.evaluate(x_test, y_test, verbose=0)
print("EfficientNetB7 (frozen) test accuracy:", test_acc_r)
print("EfficientNetB7 (frozen) test loss    :", test_loss)


EfficientNetB7 (frozen) test accuracy: 0.917900025844574
EfficientNetB7 (frozen) test loss    : 0.25737297534942627


In [ ]:
# -----------------------------
#Fine-tune last layers
# -----------------------------
effnet_base.trainable = True #unfreeze
for layer in effnet_base.layers[:-150]: #freeze all except the last 150 layer 
    layer.trainable = False

print("Trainable layers in backbone:", sum(l.trainable for l in effnet_base.layers), "/", len(effnet_base.layers))

effnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

history_ft = effnet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1,
    callbacks=callbacks,
)



Trainable layers in backbone: 150 / 814
Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 938s 1s/step - accuracy: 0.7710 - loss: 0.6956 - val_accuracy: 0.9126 - val_loss: 0.2758 - learning_rate: 1.0000e-05
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 881s 1s/step - accuracy: 0.8497 - loss: 0.4404 - val_accuracy: 0.9362 - val_loss: 0.2189 - learning_rate: 1.0000e-05
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 881s 1s/step - accuracy: 0.8791 - loss: 0.3621 - val_accuracy: 0.9400 - val_loss: 0.1916 - learning_rate: 1.0000e-05
EfficientNetB7 (fine-tuned) test accuracy: 0.9340999722480774
EfficientNetB7 (fine-tuned) test loss    : 0.19892647862434387


In [30]:
test_loss_ft, test_acc_ft = effnet_model.evaluate(x_test, y_test, verbose=0)
print("EfficientNetB7 (fine-tuned) test accuracy:", test_acc_ft)
print("EfficientNetB7 (fine-tuned) test loss    :", test_loss_ft)

EfficientNetB7 (fine-tuned) test accuracy: 0.9340999722480774
EfficientNetB7 (fine-tuned) test loss    : 0.19892647862434387


In [ ]:
# Collect and compare accuracies (update if you rename variables)
results = {
    "EfficientNetB7 frozen test acc": float(test_acc_r) if 'test_acc_r' in globals() else None,
    "EfficientNetB7 fine-tuned test acc": float(test_acc_ft) if 'test_acc_ft' in globals() else None,
}
for k,v in results.items():
    print(f"{k}: {v}")


EfficientNetB7 frozen test acc: 0.917900025844574
EfficientNetB7 fine-tuned test acc: 0.9340999722480774


## Resnet

In [14]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input

# -----------------------------
# 3) Build ResNet50V2 backbone (pretrained)
# -----------------------------
resnet_base = ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)
resnet_base.trainable = False  # freeze first (feature extractor)

# -----------------------------
# 4) Full model (preprocess inside model)
# -----------------------------
resnet_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(224, 224, interpolation="bilinear"),
    layers.Lambda(preprocess_input),     # IMPORTANT: correct for ResNet50V2 without it result in  bad accuarcy
    resnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10)                          # logits
], name="cifar10_resnet50v2")

resnet_model.summary()

# -----------------------------
# 5) Compile + Train (frozen backbone)
# -----------------------------
resnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1),
]

history = resnet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)



94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "cifar10_resnet50v2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing_1 (Resizing)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_1 (Lambda)               │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,585,290 (89.97 MB)

 Trainable params: 20,490 (80.04 KB)

 Non-trainable params: 23,564,800 (89.89 MB)

Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 174s 236ms/step - accuracy: 0.6762 - loss: 0.9324 - val_accuracy: 0.8704 - val_loss: 0.3650 - learning_rate: 0.0010
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 164s 232ms/step - accuracy: 0.8017 - loss: 0.5597 - val_accuracy: 0.8702 - val_loss: 0.3761 - learning_rate: 0.0010
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 164s 233ms/step - accuracy: 0.8235 - loss: 0.5063 - val_accuracy: 0.8872 - val_loss: 0.3286 - learning_rate: 5.0000e-04


In [25]:
# -----------------------------
# 6) Test / Evaluate
# -----------------------------
test_loss_res, test_acc_res = resnet_model.evaluate(x_test, y_test, verbose=0)
print("ResNet50V2 (frozen) test accuracy:", test_acc_res)
print("ResNet50V2 (frozen) test loss    :", test_loss_res)


ResNet50V2 (frozen) test accuracy: 0.879800021648407
ResNet50V2 (frozen) test loss    : 0.3448171019554138


In [26]:
# -----------------------------
#Fine-tune last layers
# -----------------------------
resnet_base.trainable = True #unfreeze
for layer in resnet_base.layers[:-30]: #freeze all except the last 30 layer (i guess)
    layer.trainable = False

print("Trainable layers in backbone:", sum(l.trainable for l in resnet_base.layers), "/", len(resnet_base.layers))

resnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

history_ft = resnet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)


Trainable layers in backbone: 30 / 190
Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 230s 312ms/step - accuracy: 0.7998 - loss: 0.5954 - val_accuracy: 0.9028 - val_loss: 0.2786
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 217s 308ms/step - accuracy: 0.8667 - loss: 0.3811 - val_accuracy: 0.9150 - val_loss: 0.2430
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 217s 308ms/step - accuracy: 0.8903 - loss: 0.3218 - val_accuracy: 0.9232 - val_loss: 0.2228
ResNet50V2 (fine-tuned) test accuracy: 0.9196000099182129
ResNet50V2 (fine-tuned) test loss    : 0.2317931205034256


In [29]:

test_loss_r_ft, test_acc_r_ft = resnet_model.evaluate(x_test, y_test, verbose=0)
print("ResNet50V2 (fine-tuned) test accuracy:", test_acc_r_ft)
print("ResNet50V2 (fine-tuned) test loss    :", test_loss_r_ft)

ResNet50V2 (fine-tuned) test accuracy: 0.9196000099182129
ResNet50V2 (fine-tuned) test loss    : 0.2317931205034256


---

# Custom CNN

In [22]:
# -----------------------------
# 3) Build Custom CNN Model
# ----------------------------- 


custom_cnn_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    
    # Rescaling pixels to [0, 1] (Custom CNNs need manual scaling)
    layers.Rescaling(1./255),
    
    # Feature Extraction Layers
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    
    # Classification Head
    layers.Dense(128, activation='relu'),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(10) # Logits
], name="cifar10_custom_cnn")

custom_cnn_model.summary()

# -----------------------------
# 4) Compile + Train
# -----------------------------
custom_cnn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

history = custom_cnn_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=10, # Custom CNNs usually need more epochs than transfer learning
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

Model: "cifar10_custom_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_5 (Rescaling)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 8, 8, 128)      │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │        81,930 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,642 (428.29 KB)

 Trainable params: 109,642 (428.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.3718 - loss: 1.7408 - val_accuracy: 0.5586 - val_loss: 1.2542 - learning_rate: 0.0010
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5476 - loss: 1.2767 - val_accuracy: 0.5958 - val_loss: 1.1756 - learning_rate: 0.0010
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5970 - loss: 1.1496 - val_accuracy: 0.6438 - val_loss: 1.0283 - learning_rate: 0.0010
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6315 - loss: 1.0601 - val_accuracy: 0.6770 - val_loss: 0.9387 - learning_rate: 0.0010
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.6398 - loss: 1.0225 - val_accuracy: 0.6650 - val_loss: 0.9845 - learning_rate: 0.0010
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6753 - loss: 0.9303 - val_accuracy: 0.6926 - val_loss: 0.8962 - learning_rate: 5.0000e-04
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.6831 - loss: 0.908

In [24]:
# -----------------------------
# 6) Test / Evaluate
# -----------------------------

test_loss_cnn, test_acc_cnn = custom_cnn_model.evaluate(x_test, y_test, verbose=0)

print("Custom CNN Test Accuracy:", test_acc_cnn)
print("Custom CNN Test Loss    :",test_loss_cnn)

Custom CNN Test Accuracy: 0.7039999961853027
Custom CNN Test Loss    : 0.8593738079071045


---

# 3. Then compare its performance with ResNet and the custom CNN.


In [31]:
print("Custom CNN Test Accuracy:", test_acc_cnn)

print("ResNet50V2 (frozen) test accuracy:", test_acc_res)
print("ResNet50V2 (fine-tuned) test accuracy:", test_acc_r_ft)

print("EfficientNetB7 (frozen) test accuracy:", test_acc_r)
print("EfficientNetB7 (fine-tuned) test accuracy:", test_acc_ft)



Custom CNN Test Accuracy: 0.7039999961853027
ResNet50V2 (frozen) test accuracy: 0.879800021648407
ResNet50V2 (fine-tuned) test accuracy: 0.9196000099182129
EfficientNetB7 (frozen) test accuracy: 0.6873000264167786
EfficientNetB7 (fine-tuned) test accuracy: 0.9340999722480774


---


### Questions:

#### - Which model achieved the highest accuracy?
EfficientNetB7 achieved the highest accuracy (93.4%)
#### - Which model trained faster?
The custom CNN was the fastest trained model even with more epoc.
#### - How might the architecture explain the differences?
Custom CNN: Simple and fast but lacks the pre-learned feature hierarchy of ImageNet-trained models.
ResNet: Uses Residual (Skip) Connections to train deep networks effectively.
EfficientNet: Uses MBConv blocks and specialized scaling to maximize accuracy, though its 800+ layers make it computationally heavy.